In [1]:
%pylab inline
import sys

Populating the interactive namespace from numpy and matplotlib


# The total chromaticity is given by :

# $ \xi_{tot} = \frac{1}{4\pi} \oint [ m (s)D(s)+k(s)] \beta(s) ds $

## Where m is the strength of the sextupole; D is the dispersion function; k is the quadrupole strength; $\beta$ is the beta function 

## There exists a linear relationship between the chromaticity and the sextupole strength, and the slope is proportional to the product of D and $\beta$

$  $

## $\begin{cases} \begin{align*} \xi_x &= \sum_im_i \Delta x_i + x_0 \\
                              \xi_y &= \sum_i n_i \Delta x_i + y_0 \end{align*}\end{cases}$ 


##  Where $\xi_x$, $\xi_y$ is the first order chromaticity of the original lattice 

## $x_i$ is the variation of the strength of sextupole

## $m_i$, $ n_i$ only depends on local optical functions

## $x_0$, $y_0$ is the first order chromaticity of the rotator lattice before fixing the chrom

$   $ 

$ $

In [16]:
ss = array([-3.57664, 0.8176, -3.60689, -1.7296, 0.82926, -1.69482, 0.50485067061]) # original sexutople strength

In [17]:
m = array([0.699, 6.3548, 0.6984, 0.7042, 6.3707, 0.699, 6.3643]) #coefficient of the first equation

n = array([-1.7957, -1.9373, -1.7892, -1.7899, -1.9367, -1.7831, -1.928]) # coefficient of the second equation

CX = 1.593508     #orginal chrom.x
CY = 1.622865      #orginal chrom.y

x0 = -1.190362 
y0 = -6.084533 

In [18]:
def cx(x):         #compute chrom.x
    return dot(m,x) + x0     

def cy(x):         #compute chrom.y
    return dot(n,x) + y0    

$  $

# let $\xi'_x = \xi_x - x_0 $, and  $\xi'_y = \xi_y - y_0 $

## $\begin{cases} \begin{align*} \xi'_x &= m_1 x_1+m_2 x_2 + ...+ m_N x_N  \\
                              \xi'_y &= n_1 x_1+n_2 x_2 + ...+ n_N x_N  \end{align*}\end{cases}$ 

# N variables to be determined and exists 2 equations, so there are N-2 free variables
                              
## Manipulate the second equation, find $x_1$ as a function of other variables

## $x_1 = \frac{\xi'_y - n_2 x_2 - ...- n_N x_N}{n_1}$, subsitute it into the first equation 

## $\xi'_x - \frac{m_1 \xi'_y}{n_1} = (m_2-\frac{m_1 n_2}{n_1}) x_2 +(m_3-\frac{m_1 n_3}{n_1}) x_3 + ...+ (m_N-\frac{m_1 n_N}{n_1}) x_N$

In [19]:
coeff = array([m[i]-m[0]*n[i]/n[0] for i in arange(1,len(m))])   # coefficients of equation above

print('the coefficients are :' ,coeff)

the coefficients are : [5.60068033e+00 1.93021106e-03 7.45772679e-03 5.61681388e+00
 4.90471682e-03 5.61380047e+00]


In [20]:
def func(X0,Y0,x): # function to comptue x1 and x2; x is the array contains 4 free variables (x3,x4,x5,x6)
    Xp = CX - X0   #Chrom'.x
    Yp = CY - Y0   #Chrom'.y
    lhs = Xp - m[0]*Yp/n[0]                 # compute LHS in the equation above
    x2 = (lhs-dot(coeff[1:],x))/coeff[0]    # variable x2
    x1 = (Yp - n[1]*x2- dot(n[2:],x))/n[0]  # variable x1
    return x1, x2

# To solve for the equation at minimum strength:

## Step 1: First generate a data array for N-2 free variables within a resonable range

## Step 2: Pick a number from each data array to generate a data group for those N-2 variables $\{x_3, x_4, ..., x_N\}$

## Step3: Compute $x_1, x_2$ and only record solutions which are in the desired range

## Step4: Find the maximum strength in each solution array {$x_1,x_2,... ,x_N$}

## Step5: make a comparison between the maximum value of each solution array  to find the minimum, which is the expected solution

## Step6: Do iterations, gradually narrow dataset range util a global minimum is found
## e.g. let's say, in first iteration, find $x_3$ = 3.1. The initial data range of $x_3$ is $[-5.5,5.5]$, now  add $\pm 0.5$ to 3.1.  Narrow this range to $[2.6,3.6]$, and  use it for the next iteration.

## Optimizer   

In [21]:
def multsolver(X0,Y0,a,b,c,d,f):

    xxi = []
    
    for i in range(num):
        for j in range(num): 
            for k in range(num):
                for l in range(num):
                    for m in range(num):
                        xx = array([a[i],b[j],c[k],d[l],f[m]]) # record free variables
                        x1 = func(X0,Y0,xx)[0]      # compute x1
                        x2 = func(X0,Y0,xx)[1]      # compute x2
                                    
                        if abs(x1) <=3 and 0<x2 <= 1 and 0<xx[2]<=1:
                            xs = concatenate((x1,x2),axis=None) # write x1, x2 into one array
                            xxi.append(concatenate((xs,xx),axis=None))#generate the solution array 
                        
    xxi = array(xxi)
    
    xxim = []
    for i in range(len(xxi)):
        xxim.append(max(abs(xxi[i]+ss))) # find the maxium value of N variables in each solution
    xxim = array(xxim)
    
    if len(xxim)==0:
        sys.exit('The optimizer can not find a solution in this given range, please increase the maximum strength')
        # if no solution, increase the max 
    nm = where(xxim == min(xxim))[0][0]      # find where the minimum solution is
    return xxi[nm]         # return solution

In [22]:
def opt(x0,y0,mm,ni,num):   # sextupole strength optimizer
    
    ama =  mm   ## max of the data 
    ami = -mm   ## min of the data 
    bma =  mm
    bmi =  mm
    cma =  mm
    cmi = -mm
    dma =  mm
    dmi = -mm
    fma =  mm
    fmi = -mm

    
    sa = []  # initialize solution array
    msa = [] # initialize maximum strength array
    
    for i in range(ni):
        a = linspace(ami,ama,num)     # generating data within the required range 
        b = linspace(bmi,bma,num)
        c = linspace(cmi,cma,num)
        d = linspace(dmi,dma,num)
        f = linspace(fmi,fma,num)
        
        s = multsolver(x0,y0,a,b,c,d,f) # compute x1,x2
    
        ms = max(abs(s+ss))            # find the max in solution array
    
        ra = 1- i/10                    # narrowing the variation range of data array
    
        if ra<0.1:                     # set minimum variation range
            ra=0.1

        if i!= 0 and ms >= msa[-1]:    # if iteration step finds a bigger strength, 
                                       #go back to previous solution and increase num
            num += 1
        
            ama = sa[-1][2]+ra
            ami = sa[-1][2]-ra
        
            bma = sa[-1][3]+ra
            bmi = sa[-1][3]-ra
        
            cma = sa[-1][4]+ra
            cmi = sa[-1][4]-ra
        
            dma = sa[-1][5]+ra
            dmi = sa[-1][5]-ra
            
            fma = sa[-1][6]+ra
            fmi = sa[-1][6]-ra
            
        else:
            ama = s[2]+ra
            ami = s[2]-ra
        
            bma = s[3]+ra
            bmi = s[3]-ra
        
            cma = s[4]+ra
            cmi = s[4]-ra
        
            dma = s[5]+ra
            dmi = s[5]-ra
            
            fma = s[6]+ra
            fmi = s[6]-ra
            
        
        if num >= 15:    #insurance of efficiency
            num = 15
            
        sa.append(s)
        msa.append(ms)

        print('iteration step: %s'%(i+1))
        print('the max strength is:',ms)
        print(s+ss)
        print(num)
        print('')
        print('')
    
    sa = array(sa)
    msa = array(msa)

    nm = where(msa ==min(msa))[0][0]      # find the where the minimum solution is

    xs = sa[nm]
    print('the iretation step reached mininum:',nm+1)
    print('the max strength in the solution is:',min(msa))
    return xs

In [23]:
mm = 2.7   # max variation of sextupole strength
ni = 15    # number of iterations to do
num  = 8   # number of data to be generated

In [24]:
solution = opt(x0,y0,mm,ni,num)

iteration step: 1
the max strength is: 6.30689
[-6.29833758  1.07660189 -6.30689     0.9704      1.21497429 -4.39482
  0.89056496]
8


iteration step: 2
the max strength is: 5.39482
[-5.31076851  1.65130862 -5.30689    -0.0296      1.07211714 -5.39482
  0.46199353]
8


iteration step: 3
the max strength is: 5.1752932604902595
[-5.17529326  0.87886978 -4.92117571 -0.9296      0.94354571 -5.00910571
  1.36199353]
8


iteration step: 4
the max strength is: 4.8334708379835565
[-4.83347084  0.87953391 -4.80689    -1.7296      1.05783143 -4.66624857
  1.24770781]
8


iteration step: 5
the max strength is: 4.566248571428572
[-4.54048242  1.68220293 -4.50689    -2.4296      0.95783143 -4.56624857
  0.54770781]
8


iteration step: 6
the max strength is: 4.421175714285715
[-4.28338455  1.68279326 -4.42117571 -3.0296      0.87211714 -4.30910571
  0.6334221 ]
8


iteration step: 7
the max strength is: 4.2068900000000005
[-4.20664541  0.82385459 -4.20689    -3.5296      1.22926    -4.09482
  1.1334

In [25]:
ss #original sextupoles

array([-3.57664   ,  0.8176    , -3.60689   , -1.7296    ,  0.82926   ,
       -1.69482   ,  0.50485067])

In [26]:
solution #variation of the sextupoles strength

array([-0.43599239,  0.3505346 , -0.40357143, -2.275     ,  0.16428571,
       -2.31785714,  0.52142857])

In [27]:
solution+ss #solution array 

array([-4.01263239,  1.1681346 , -4.01046143, -4.0046    ,  0.99354571,
       -4.01267714,  1.02627924])

# Orginal chromaticity  $\xi_x = 1.593508$   

In [28]:
cx(solution) # testing the solution

1.5935079999999986

# Orginal chromaticity $ \xi_y = 1.622865$

In [29]:
cy(solution)

1.622865000000001

# If the chrom matches above, use this solution as a initial guess to runing a optimization in tao.